In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import math
from datetime import datetime
root=''
try:
  from google.colab import drive
  drive.mount('/content/drive')
  root='./drive/MyDrive/gnn/data/'
except:
  pass
device='cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

In [ ]:
filename=root+'data_1354ac_2022/ieee1354_ISF.txt'
S_isf=pd.read_table(filename,sep=',',header=None).to_numpy() # ISF matrix
filename=root+'data_1354ac_2022/ieee1354_fmax.txt'
f_max=pd.read_table(filename,sep=',',header=None).to_numpy() # flow limit
n_line = np.size(S_isf,0)
S = torch.from_numpy(S_isf).to(device) # ISF
f_max = torch.from_numpy(f_max).to(device) # flow limit
print(S.shape,f_max.shape,torch.min(f_max))

x=np.load(root+'data_1354ac_2022/ac1354_x_04022022.npy')
y=np.load(root+'data_1354ac_2022/ac1354_y_04022022.npy')
W=np.load(root+'data_1354ac_2022/ac1354_w.npy')
print(x.shape,y.shape)
print(np.max(x),np.min(x),np.max(y),np.min(y))

# scaling on voltage
vy_deviation = 0.8
vy_scale = 1000
y[:,1,:] = (y[:,1,:] - vy_deviation) * vy_scale
print('voltage range(scaled):',np.min(y[:,1,:]),np.max(y[:,1,:]))

# scaling on price
pi_deviation = 150
y[:,0,:] = y[:,0,:] + pi_deviation


# filter out extreme points in price
y_sort_arg = np.argsort(np.amax(np.abs(y[:,0,:]),axis=0)) # max extreme
y_sort_arg1 = np.argsort(np.amin(y[:,0,:],axis=0),axis=0) # min extreme

del_idx0 = []
del_num = 500
for i in range(del_num):
  del_idx0.append(y_sort_arg[-i])
  del_idx0.append(y_sort_arg1[i])
print(del_idx0)
del_idx = [] # keep only non-repeated
[del_idx.append(x) for x in del_idx0 if x not in del_idx]
del_idx = np.sort(del_idx)
# delete extreme points
print('price range old:',np.min(y[:,0,:]),np.max(y[:,0,:]))
print('voltage range old:',np.min(y[:,1,:]),np.max(y[:,1,:]))
x = np.delete(x, del_idx, axis=2)
y = np.delete(y, del_idx, axis=2)


# generate training/test dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x.transpose(),y.transpose(),test_size=0.2, random_state=18)
x_train=x_train.transpose()
x_test=x_test.transpose()
y_train=y_train.transpose()
y_test=y_test.transpose()
print('Training data size:',x_train.shape)
print('Training label size:',y_train.shape)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, device='cpu'):
        self.features=torch.from_numpy(np.transpose(features)).float().to(device)
        self.labels=torch.from_numpy(np.transpose(labels)).float().to(device)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
        # Select sample
        X = self.features[idx]  # shape = (24,)
        y = self.labels[idx]    # shape = (24,)
        X = torch.reshape(X,(6,1354))
        y = torch.reshape(y,(2,1354))
        return X, y
params = {'batch_size': 256,
          'shuffle': True,
          'num_workers': 0}
# Dataset Generators
training_set = Dataset(features=x_train,labels=y_train,device=device)
training_generator = torch.utils.data.DataLoader(training_set,**params)
validation_set = Dataset(features=x_test,labels=y_test,device=device)
validation_generator = torch.utils.data.DataLoader(validation_set,**params)

print('price range new:',np.min(y[:,0,:]),np.max(y[:,0,:]))
print('voltage range new:',np.min(y[:,1,:]),np.max(y[:,1,:]))

In [ ]:
# fig2 = plt.figure(figsize=(16, 8))
# flat_list = list(np.concatenate(y[:,1,:]).flat)
# flat_list3 = list(np.concatenate(y[:,0,:]).flat)
# plt.subplot(1,2,1)
# plt.hist(flat_list,bins = 100,label = 'voltage')
# plt.subplot(1,2,2)
# # plt.hist(flat_list3,range=[-2000, 2000],bins = 100,label = 'price')
# plt.hist(flat_list3,bins = 100,label = 'price')
# plt.show()

* Normalize the power of Laplacian

In [ ]:
W1 = W.copy()
W1 = np.asarray(W1)
# print(np.asarray([[2,3],[4,7]])/5)
W1 = W1 / 1
W2 = W1.copy()
print(type(W))
lam,v = np.linalg.eig(W1)
print(lam[0])
for i in range(5):
  W1 = np.matmul(W1,W2) / np.max(np.matmul(W1,W2))
  # print(W1)
  lam,v = np.linalg.eig(W1)
  print(lam[0])#/10**(i+1))

In [ ]:
def numparam(in_feats,out_feats): return 4*476+in_feats*out_feats+out_feats
indexes=[1,10,15,15,10,10,1]
total=0
for temp in range(len(indexes)-1):
  total+=numparam(indexes[temp],indexes[temp+1])
print(total)

In [ ]:
from torch.autograd import Variable
# One layer Graph convolution from nodes to edges
class Graph_convolution_v2v_W(nn.Module):
    def __init__(self,in_features,out_features,W,n,bias=True):
        super(Graph_convolution_v2v_W,self).__init__()
        self.register_buffer('w',torch.from_numpy(W.transpose()).float())
        self.register_buffer('n0',torch.tensor(n))
        self.mapping=nn.Parameter(torch.Tensor(n,W.shape[0],W.shape[1]))
        torch.nn.init.xavier_uniform_(self.mapping.data)
        self.scale0=nn.Parameter(torch.Tensor(out_features,in_features))
        torch.nn.init.xavier_uniform_(self.scale0.data)
        self.scale=nn.Parameter(torch.Tensor(n-1,out_features,out_features))
        torch.nn.init.xavier_uniform_(self.scale.data)
        self.bias=nn.Parameter(torch.Tensor(out_features,1))
        torch.nn.init.xavier_uniform_(self.bias.data)

    def forward(self,input):
        h=torch.mul(self.mapping[0,:,:],self.w) # element-wise
        h=torch.matmul(input,h) # transpose due to batch generator
        h=torch.matmul(self.scale0,h)
        for i in range(self.n0 - 1):
          W1 = torch.matrix_power(self.w,i+2)/torch.max(torch.matrix_power(self.w,i+2))
          h1=torch.mul(self.mapping[i,:,:],W1)
          h=torch.matmul(h,h1) # transpose due to batch generator 
          h=torch.matmul(self.scale[i,:,:],h)
        return h+self.bias 

# GNN using DGL v2v graph convolution and our own v2e graph convolution
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, W, num_bus, n_hop):
        super(GCN, self).__init__()
        self.conv_v2v1=Graph_convolution_v2v_W(in_feats,hidden_size[0],W,n_hop)
        self.conv_v2v2=Graph_convolution_v2v_W(hidden_size[0],hidden_size[1],W,n_hop)
        self.conv_v2v3=Graph_convolution_v2v_W(hidden_size[1],hidden_size[2],W,n_hop)
        self.conv_v2v4=Graph_convolution_v2v_W(hidden_size[2],hidden_size[3],W,n_hop)
        self.conv_v2v5=Graph_convolution_v2v_W(hidden_size[3],hidden_size[4],W,n_hop)
        self.conv_v2v6=Graph_convolution_v2v_W(hidden_size[4],hidden_size[5],W,n_hop)
        # self.conv_v2v7=Graph_convolution_v2v_W(hidden_size[5],hidden_size[6],W,n_hop)
        # self.conv_v2v8=Graph_convolution_v2v_W(hidden_size[6],hidden_size[7],W,n_hop)
        self.lin_output = nn.Linear(num_bus,num_bus)
         
    def forward(self, inputs):
        # m = nn.ELU()
        # m = nn.LeakyReLU()
        # m = nn.Tanh()
        m = nn.ReLU()
        h=self.conv_v2v1(inputs)
        h=m(h)
        h=self.conv_v2v2(h)
        h=m(h)
        h=self.conv_v2v3(h)
        h=m(h)
        h=self.conv_v2v4(h)
        h=m(h)
        h=self.conv_v2v5(h)
        h=m(h)
        h=self.conv_v2v6(h)
        h=torch.relu(h)
        h = self.lin_output(h)
        # time.sleep(5)
        return h
n_bus=x.shape[0]
# w_params=[10,15,10,1]
w_params=[5,10,10,5,5,2]
n_hop = 2
# W = W / 10 # normalize
net=GCN(6,w_params,W,n_bus, n_hop) # Laplacian
net=net.to(device)

# set loss_func & optimizer

# optimizer=torch.optim.Adam(net.parameters(),lr=0.01)
optimizer=torch.optim.Adam(net.parameters())
loss_optm=[]
loss_val=[]
print(net)
print('number of params: %d'%(sum(temp.numel() for temp in net.parameters() if temp.requires_grad)))

In [ ]:
# Number of parameters in net
n_params = 0
n_params_trainable = 0
for temp in net.parameters():
    # print(temp.shape[0])
    # print(temp.shape)
    n_params += temp.numel()
    if abs(temp.shape[0] - n_bus) > 0:
      n_params_trainable += temp.shape[0] * temp.shape[1]
    else:
      n_params_trainable += n_bus + 2 * n_line
print('Number of GNN parameters:',n_params)
print('Number of effective GNN parameters:',n_params_trainable)

In [ ]:
# threshold function for p_g
class my_gen_pred_binary(nn.Module):
  def __init__(self):
    super(my_gen_pred_binary,self).__init__()
  def forward(self,x,thresh):
    right_thresh=thresh.clone().detach().requires_grad_(True).double()
    left_thresh=torch.tensor(0).double()
    x=x.double()
    output = torch.sigmoid(left_thresh - x)
    output = torch.mul(output,left_thresh - x) + x
    output = torch.sigmoid(output - right_thresh)
    output = torch.mul(output,output - right_thresh) + right_thresh
    return output

In [ ]:
## params needed for S calculation
# line parameters
filename1 = root+'data_1354ac_2022/ieee1354_lineloc.txt'
filename2 = root+'data_1354ac_2022/ieee1354_lineparams_tap.txt'
filename3 = root+'data_1354ac_2022/ieee1354_Bbus.txt'
filename4 = root+'data_1354ac_2022/ieee1354_ieee1354_J1.txt'
filename5 = root+'data_1354ac_2022/ieee1354_ieee1354_theta_nominal.txt'
filename6 = root+'data_1354ac_2022/ieee1354_ieee1354_pinj_nominal.txt'
# incidence info
line_loc = pd.read_table(filename1,sep=',',header=None).to_numpy()
# r, x, shunt, S_max
line_params = pd.read_table(filename2,sep=',',header=None).to_numpy()
B_mat=pd.read_table(filename3,sep=',',header=None).to_numpy()
# params for fast decoupled power flow
J_pf=pd.read_table(filename4,sep=',',header=None).to_numpy()
theta0_nominal=pd.read_table(filename5,sep=',',header=None).to_numpy()
pinj0_nominal=pd.read_table(filename6,sep=',',header=None).to_numpy()

# remove transformers
n_line = 1751 # the rest are trabsformers
line_loc = line_loc[:n_line,:].copy()
line_params = line_params[:n_line,:].copy()

B_r = np.delete(B_mat,639,axis=0) # ref bus
B_r = np.delete(B_r,639,axis=1)
Br_inv = np.linalg.inv(B_r)
R_line = line_params[:,0].copy()
X_line = line_params[:,1].copy()
B_shunt = line_params[:,2].copy()
Z_line = R_line + 1j * X_line 
Y_line = 1 / Z_line
G_line = np.real(Y_line)
B_line = np.imag(Y_line)
a = np.ones(n_line)
f_max = f_max[:n_line,:]
print(f_max.shape)

Jpf_inv = np.linalg.inv(J_pf)
theta0_nominal = theta0_nominal/180*math.pi
# pinj0_nominal already in p.u.

# params to tensor and GPU
G_line_tensor = torch.from_numpy(G_line).to(device) # conductance
B_line_tensor = torch.from_numpy(B_line).to(device) # susceptance
B_shunt_tensor = torch.from_numpy(B_shunt/2).to(device) # conductance
Br_inv_tensor = torch.from_numpy(Br_inv).to(device) # reduced Bbus matrix
a_tensor = torch.from_numpy(a).double().to(device) # line/transformer
W_tensor = torch.from_numpy(W).float().to(device)
Jpf_tensor = torch.from_numpy(Jpf_inv).to(device) # FDPF Jacobian
theta0_tensor = torch.from_numpy(theta0_nominal).to(device) # nominal angle
pinj0_tensor = torch.from_numpy(pinj0_nominal).to(device) # p_inj 

vy_scale_tensor = torch.from_numpy(np.asarray(vy_scale)).double().to(device)
vy_deviation_tensor = torch.from_numpy(np.asarray(vy_deviation)).double().to(device)
pi_deviation_tensor = torch.from_numpy(np.asarray(pi_deviation)).double().to(device)

print(line_loc.shape,line_params.shape)
print(np.max(line_params[:,4]),line_params[1750,4])

In [ ]:
class loss_func:
    def __init__(self,s_max,G_line,B_line,B_shunt,Br_inv,a,line_loc,Jpf,theta_n,pinj0,vy_scale,vy_deviation,pi_deviation):
      self.s=s_max
      self.g=G_line
      self.b=B_line
      self.c=B_shunt
      self.r=Br_inv
      self.a=a
      self.J1=Jpf
      self.t0=theta_n
      self.p0=pinj0
      self.vy_s=vy_scale
      self.vy_d=vy_deviation
      self.vy_pd=pi_deviation
      self.mse=nn.MSELoss() # MSE loss
      # hyper params for loss items
      self.lmda1=torch.tensor(10).to(device) # V MSE 
      self.lmda2=torch.tensor(1).to(device) # pi MSE 
      self.lmda3=torch.tensor(0.1).to(device) # v l_inf
      self.lmda4=torch.tensor(0).to(device) # s feasibility
      self.lmda5=torch.tensor(0).to(device) # pi l_inf
      # self.thres=torch.tensor(0.97).to(device)

      self.line_loc=line_loc.copy()
      self.binary_cell=my_gen_pred_binary()
    def calc(self,pred,label,x):
      mse_v=self.mse(pred[:,1,:],label[:,1,:])
      mse_p=self.mse(pred[:,0,:],label[:,0,:])
      linf_v=(pred[:,1,:]-label[:,1,:]).norm(p=float('inf'))
      linf_p=(pred[:,0,:]-label[:,0,:]).norm(p=float('inf'))

      label_pred=pred[:,0,:] - self.vy_pd # price 
      v_pred=pred[:,1,:]
      label1=torch.squeeze(label_pred,1).transpose(0,1)
      p_max=x[:,0,:].transpose(0,1)-x[:,1,:].transpose(0,1)
      quadratic_b=x[:,4,:].transpose(0,1)
      quadratic_a=x[:,5,:].transpose(0,1)
      quadratic_center=(label1-quadratic_b)/(quadratic_a+1e-10)/2
      p_inj=self.binary_cell(quadratic_center,p_max)
      bus_inj=p_inj+x[:,1,:].transpose(0,1)
      p_inj_r=torch.cat((bus_inj[:639,:],bus_inj[640:,:]))/100
      # theta0=torch.matmul(self.r,p_inj_r) # dcpf model
      # fdpf model start
      # print(self.J1.shape,p_inj_dev.shape,self.t0.shape)
      p_inj_dev=p_inj_r-self.p0
      theta0=torch.matmul(self.J1,p_inj_dev) # fdpf model
      # fdpf model end
      ref_ang=torch.zeros(1,theta0.shape[1]).to(device)
      theta=torch.cat([theta0[:639,:],ref_ang,theta0[639:,:]],0)+self.t0 # deviation +nominal
      v_pred=v_pred.transpose(0,1)/self.vy_s+self.vy_d

      # s penalty
      theta1=theta[self.line_loc[:,0]-1,:]
      theta2=theta[self.line_loc[:,1]-1,:]
      V1=(v_pred[self.line_loc[:,0]-1,:]).double()
      V2=(v_pred[self.line_loc[:,1]-1,:]).double()
      f_p=(self.a*self.g*(V1*V1).T)-self.a*((V1*V2).T)*(self.g*torch.cos(theta1-theta2).T+self.b*torch.sin(theta1-theta2).T)
      f_p=f_p.T
      f_q=-self.a*(V1.T)*(self.a*V1.T)*(self.b+self.c/2)+self.a*((V1*V2).T)*(self.b*torch.cos(theta1-theta2).T-self.g*torch.sin(theta1-theta2).T)
      f_q=f_q.T
      s_pred=torch.sqrt(f_p*f_p+f_q*f_q)*100
      s_penalty=torch.sigmoid(s_pred-self.s)+torch.sigmoid(-s_pred-self.s)
      s_total=torch.sum(s_penalty)

      # sji penalty
      theta1=theta[self.line_loc[:,1]-1,:]
      theta2=theta[self.line_loc[:,0]-1,:]
      V1=(v_pred[self.line_loc[:,1]-1,:]).double()
      V2=(v_pred[self.line_loc[:,0]-1,:]).double() 
      fji_p=(self.a*self.g*(V1*V1).T)-self.a*((V1*V2).T)*(self.g*torch.cos(theta1-theta2).T+self.b*torch.sin(theta1-theta2).T)
      fji_p=fji_p.T
      fji_q=-self.a*(V1.T)*(self.a*V1.T)*(self.b+self.c/2)+self.a*((V1*V2).T)*(self.b*torch.cos(theta1-theta2).T-self.g*torch.sin(theta1-theta2).T)
      fji_q=fji_q.T
      sji_pred=torch.sqrt(fji_p*fji_p+fji_q*fji_q)*100
      sji_penalty=torch.sigmoid(sji_pred-self.s)+torch.sigmoid(-sji_pred-self.s)
      sji_total=torch.sum(sji_penalty)

      return self.lmda1*mse_v+self.lmda2*mse_p+self.lmda3*linf_v+self.lmda5*linf_p+self.lmda4*s_total+self.lmda4*sji_total


my_loss=loss_func(f_max,G_line_tensor,B_line_tensor,B_shunt_tensor,Br_inv_tensor,a_tensor,line_loc,Jpf_tensor,theta0_tensor,pinj0_tensor,vy_scale_tensor,vy_deviation_tensor,pi_deviation_tensor)

In [ ]:
optimizer=torch.optim.Adam(net.parameters())

# load the pretrained parameters
# path=root+'./data_1354ac_2022/ac_v_S_feasgnn_0318.pickle'
# path=root+'data_1354ac_2022/gnn_trained_ac1354.pickle' # unregularized
# path=root+'data_1354ac_2022/feasgnn_trained_ac1354_04042022_copy3.pickle' # regularized
# path=root+'data_1354ac_2022/feasgnn_trained_ac1354_04022022cold_unreg2.pickle' # unregularized good
# try: net.load_state_dict(torch.load(path))
# except: pass
try: 
  net.load_state_dict(torch.load(path))
  print('params loaded')
except: 
  print('cold start')
  pass

## Training
t0=time.time()
max_epochs=50
eval_epoch=5

# earlystopping
tolerance=8
min_delta=1e-3
previous=0

for epoch in range(max_epochs):
  # training loop
  train_loss=0.0
  for local_batch,local_label in training_generator:
    optimizer.zero_grad() # clear the past gradient
    logits=net(local_batch)
    loss=my_loss.calc(logits,local_label,local_batch)
    loss.backward()
    train_loss+=loss.item()
    optimizer.step() # update parameters of net
  loss_optm.append(train_loss/len(training_generator.dataset))
  print("Epoch %d | Training loss: %.4f"%(epoch,train_loss/len(training_generator.dataset)))
  # eval
  if (epoch+1)%eval_epoch==0:
    net.eval()
    eval_loss=0.0
    for local_batch,local_label in validation_generator:
      logits=net(local_batch)
      loss=my_loss.calc(logits,local_label,local_batch)
      eval_loss+=loss.item()
    eval_avg=eval_loss/len(validation_generator.dataset)
    if (epoch==0): previous=eval_avg
    else:
      if previous-eval_avg<min_delta: tolerance-=1
      if tolerance==0:
        break
        # pass
      previous=eval_avg
    print("Epoch %d | Eval loss: %.4f" % (epoch, eval_avg))
    loss_val.append([epoch, eval_loss/len(validation_generator.dataset)])
    net.train()
  # validation loop
  with torch.set_grad_enabled(False):
      pass
t1=time.time()
print("Training time:%.4fs"%(t1-t0))

In [ ]:
# logits[0:2,:,:]
# W
timestamp=datetime.now().strftime('%m%d%H%M')
path=root+'data_1354ac_2022/feasgnn0411_%s.pickle'%(timestamp)
print(path)
torch.save(net.state_dict(),path)

In [ ]:
import matplotlib.pyplot as plt

val_len = len(loss_val)
print(val_len)
val_plt = np.zeros((2,val_len))
for i in range(val_len):
  val_plt[0,i] = loss_val[i][0]
  val_plt[1,i] = loss_val[i][1]

# plt.figure()
# plot_idx = np.arange(np.size(loss_optm))
# # plt.plot(plot_idx,loss_optm,lw=2,label='training loss')
# # plt.plot(val_plt[0,:],val_plt[1,:],lw=2,label='validation loss')
# plt.plot(plot_idx[5:-1],loss_optm[5:-1],lw=2,label='training loss')
# plt.plot(val_plt[0,1:],val_plt[1,1:],lw=2,label='validation loss')
# plt.yscale("log")
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.legend()
# plt.show(block=False)

# Evaluate the model w/ validation set

In [ ]:
n_test = 2000

x_test_feed = torch.from_numpy(np.transpose(x_test)).float()
x_test_feed = x_test_feed#.transpose(1,2)
x_test_feed = x_test_feed.to(device)
print('Validation dataset size:',x_test_feed.shape)
print('Number of validation set: ',n_test)
y_pred = net(x_test_feed)

* Visualization




 - Visualize errors

In [ ]:
y_pred1 = y_pred.cpu().detach()
y_pred1 = torch.squeeze(y_pred1,1).numpy().transpose()
print(y_test.shape)

In [ ]:
y_pred1[:,1,:] = y_pred1[:,1,:] / vy_scale + vy_deviation
y_test[:,1,:] = y_test[:,1,:] / vy_scale + vy_deviation

print(y_test.shape,y_pred1.shape)

In [ ]:
n_test = np.size(y_test,2)
err_L2 = np.zeros(n_test)
err_Linf = np.zeros(n_test)
for i in range(n_test):
  err_L2[i] = np.linalg.norm(y_test[:,0,i] - y_pred1[:,0,i]) / np.linalg.norm(y_test[:,0,i])
  err_Linf[i] = np.max(np.abs(y_test[:,0,i] - y_pred1[:,0,i])) / np.max(np.abs(y_test[:,0,i]))

err_L2_v = np.zeros(n_test)
err_Linf_v = np.zeros(n_test)
for i in range(n_test):
  err_L2_v[i] = np.linalg.norm(y_test[:,1,i] - y_pred1[:,1,i]) / np.linalg.norm(y_test[:,1,i])
  err_Linf_v[i] = np.max(np.abs(y_test[:,1,i] - y_pred1[:,1,i])) / np.max(np.abs(y_test[:,1,i]))

In [ ]:
print(err_L2.shape,err_Linf.shape)

In [ ]:
err_L2_mean = np.mean(err_L2)
err_Linf_mean = np.mean(err_Linf)
print('Price L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )
err_L2_mean_v = np.mean(err_L2_v)
err_Linf_mean_v = np.mean(err_Linf_v)
print('Voltage L2 mean:', err_L2_mean_v,'L_inf mean:', err_Linf_mean_v )

# fig2 = plt.figure(figsize=(16, 16))
# plt.subplot(2, 2, 1)
# # plt.hist(np.abs(ga),bins = 10)
# plt.plot(err_L2,'bo',markersize=0.5,label = 'L2 error')
# plt.plot(err_Linf,'r^',markersize=0.5,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample index')
# plt.ylabel('error')
# plt.title('Normalized sample error')
# plt.grid(True)
# # error histogram
# plt.subplot(2, 2, 2)
# plt.hist(err_L2, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# plt.hist(err_Linf, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)

# plt.subplot(2, 2, 3)
# # plt.hist(np.abs(ga),bins = 10)
# plt.plot(err_L2_v,'bo',markersize=0.5,label = 'L2 error')
# plt.plot(err_Linf_v,'r^',markersize=0.5,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample index')
# plt.ylabel('error')
# plt.title('Normalized sample error')
# plt.grid(True)
# # error histogram
# plt.subplot(2, 2, 4)
# plt.hist(err_L2_v, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# plt.hist(err_Linf_v, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)

# plt.show()

In [ ]:
print(y_pred1[:,1,:n_test].shape,y_test[:,1,:n_test].shape)
print('true range:',np.max(y_test[:,1,:n_test]),np.min(y_test[:,1,:n_test]))
print('predicted range',np.max(y_pred1[:,1,:n_test]),np.min(y_pred1[:,1,:n_test]))

# fig3 = plt.figure(figsize=(16, 8))
# flat_list1 = list(np.concatenate(y_test[:,1,:n_test]).flat)
# flat_list2 = list(np.concatenate(y_pred1[:,1,:n_test]).flat)
# plt.hist(flat_list1,bins = 100,label = 'true')

# plt.hist(flat_list2,bins = 100,label = 'pred')
# plt.legend(loc="upper right")
# plt.grid(True)
# plt.show()

In [ ]:
err_l2_new = [i for i in err_L2 if i < 1]
err_linf_new = [i for i in err_Linf if i < 1]
print(len(err_l2_new),'L2 mean:', np.mean(err_l2_new),len(err_linf_new),'L_inf mean:', np.mean(err_linf_new) )

# fig2 = plt.figure(figsize=(16, 8))
# plt.subplot(1, 2, 1)
# # plt.hist(np.abs(ga),bins = 10)
# plt.plot(err_l2_new,'bo',markersize=0.5,label = 'L2 error')
# plt.plot(err_linf_new,'r^',markersize=0.5,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample index')
# plt.ylabel('error')
# plt.title('Normalized sample error w/out >1')
# plt.grid(True)
# # error histogram
# plt.subplot(1, 2, 2)
# plt.hist(err_l2_new, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# plt.hist(err_linf_new, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error  w/out >1')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

# Predict generation using $\pi$
* Using predicted $\pi$ and find the active constraints in $p_G(i)$
* For inactive $p_G(i)$ consider other methods like power flow balance

In [ ]:
gen_limit0 = x[:,4,:].copy() # lin cost
print(gen_limit0.shape)

gen_idx = []
gen_idx = np.arange(n_bus)
print(type(gen_idx),len(gen_idx),gen_idx)

In [ ]:
# # get the generator index
# # generator cost data
# gen_cost0
# # generator limit data
# gen_limit0
# # LMP data
# lmp_data
# # generation data
# gen_data

load_data = x.copy()
load_data = load_data[:,:]
print(load_data.shape)
n_sample = np.size(load_data,2)

# predict the corredponding LMP
x_val_feed = torch.from_numpy(np.transpose(load_data)).float()
batch_size=2000
y_pred1=[]
with torch.no_grad():
  for idx in range(0,x_val_feed.shape[0],batch_size):
    current_batch=x_val_feed[idx:idx+batch_size]
    current_batch=current_batch.to(device)
    current_y=net(current_batch)
    current_y=current_y.cpu().detach()
    current_y=torch.squeeze(current_y,1).numpy().tolist()
    y_pred1+=current_y
y_pred1=np.array(y_pred1).transpose()

print('Dataset size:',x_val_feed.shape)
print('Number of validation points:: ',n_sample)
print('output size',y_pred1.shape)
print('reshaped size',y_pred1.shape)

* Save results

In [ ]:
gen_cost0 = x[:,4,:].copy()
lmp_data = y[:,0,:].copy()
quadratic_a = x[:,5,:].copy()
profit_pred = y_pred1[:,0,:] - gen_cost0
print(np.min(np.abs(profit_pred)))
profit_true = lmp_data - gen_cost0
print(np.min(np.abs(profit_true)))
profit_pred=(y_pred1[:,0,:]-gen_cost0)/(quadratic_a+1e-10)/2
profit_true=(lmp_data-gen_cost0)/(quadratic_a+1e-10)/2
print(np.min(np.abs(profit_pred)))
print(np.min(np.abs(profit_true)))

In [ ]:
print(profit_pred.shape,profit_true.shape)
profit_err = profit_true - profit_pred
profit_err_l2 = np.zeros([n_sample,1])
for i in range(n_sample):
  profit_err_l2[i] = np.linalg.norm(profit_err[:,i])/np.linalg.norm(profit_true[:,i])
print(np.mean(profit_err_l2))

# fig5 = plt.figure(figsize=(16, 8))
# # error histogram
# plt.hist(profit_err_l2, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# # plt.hist(err_linf_new, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
p_pred_sort = np.reshape(profit_pred,n_bus*n_sample)
p_true_sort = np.reshape(profit_true,n_bus*n_sample)
print(p_pred_sort.shape)

In [ ]:
print(np.min(p_pred_sort),np.min(p_true_sort))

In [ ]:
# fig2 = plt.figure(figsize=(8, 8))
# plt.hist(p_pred_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. profit')
# plt.hist(p_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true profit')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('profit histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

 # Test flow feasibility

In [ ]:
# x = [load, gen_cost, gen_lim]

# binary_thres = 0.97
binary_thres_true = 1e-5
# binary_thres = gen_limit0
binary_thres = x[:,0,:].copy() # upper
binary_thres_lo = x[:,1,:].copy() # lower

gen_pred_binary_full = np.zeros((n_bus,n_sample))
gen_true_binary_full = np.zeros((n_bus,n_sample))

## nodal injection
# gen_limit0 = x[:,2,:].copy()
for i in range(n_sample):
  for j in range(len(gen_idx)):
    # predicted generator limit
    if profit_pred[gen_idx[j],i] > binary_thres[gen_idx[j],i]:
      gen_pred_binary_full[gen_idx[j],i] = binary_thres[gen_idx[j],i]
    elif profit_pred[gen_idx[j],i] < binary_thres_lo[gen_idx[j],i]:
      gen_pred_binary_full[gen_idx[j],i] = binary_thres_lo[gen_idx[j],i]
    else:
      gen_pred_binary_full[gen_idx[j],i] = profit_pred[gen_idx[j],i]
    # true generator limit
    if profit_true[gen_idx[j],i] > binary_thres[gen_idx[j],i]:
      gen_true_binary_full[gen_idx[j],i] = binary_thres[gen_idx[j],i]
    elif profit_true[gen_idx[j],i] < binary_thres_lo[gen_idx[j],i]:
      gen_true_binary_full[gen_idx[j],i] = binary_thres_lo[gen_idx[j],i]
    else:
      gen_true_binary_full[gen_idx[j],i] = profit_true[gen_idx[j],i]


# injection by generators
gen_inj=gen_pred_binary_full
gen_inj_true=gen_true_binary_full
# nodal injection
load0 = -x[:,1,:].copy() # load file
p_inj = gen_inj #- load0
p_inj_true = gen_inj_true #- load0

## injection accuracy

In [ ]:
print(p_inj_true.shape,p_inj.shape)

p_inj_true_sort = np.reshape(p_inj_true,n_bus*n_sample)
p_inj_sort = np.reshape(p_inj,n_bus*n_sample)

p_err = np.zeros(n_sample)
for i in range(n_sample):
  p_err[i] = np.linalg.norm(p_inj_true[:,i]-p_inj[:,i]) / np.linalg.norm(p_inj_true[:,i])

print('mean p_inj l2 err:',np.mean(p_err))
# fig3 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(p_inj_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. injection')
# plt.hist(p_inj_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true injection')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('injection histogram')
# plt.subplot(1,2,2)
# plt.hist(p_err, bins = 50, facecolor='b', alpha=0.75,label = 'injection err')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('error histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

* Calculate flow

In [ ]:
filename=root+'data_1354ac_2022/ieee1354_fmax.txt'
f_max1=pd.read_table(filename,sep=',',header=None).to_numpy() # flow limit

n_line = np.size(S_isf,0)
flow_est = np.zeros((n_line,n_sample))
flow_est0 = np.zeros((n_line,n_sample))

f_binary = np.zeros((n_line,n_sample))
f_binary0 = np.zeros((n_line,n_sample))

# for i in range(n_sample):
flow_est = np.dot(S_isf,p_inj)
flow_est0 = np.dot(S_isf,p_inj_true)
# f_max
# f_max_numpy = f_max.cpu().detach().numpy()
f_max_numpy = f_max1.copy()
f_binary = (np.abs(flow_est)-f_max_numpy > 0)
f_binary0 = (np.abs(flow_est0)-f_max_numpy > 0)

In [ ]:
print(f_max_numpy.shape,flow_est.shape,flow_est0.shape)

In [ ]:
f_tot_sample = n_line * n_sample
print(np.sum(f_binary),np.sum(f_binary0))
print(np.sum(f_binary)/f_tot_sample,np.sum(f_binary0)/f_tot_sample)

In [ ]:
print(n_line,n_sample,flow_est.shape)

In [ ]:
# soft threshold
f_err_est = np.abs(flow_est)-f_max_numpy
f_err_true = np.abs(flow_est0)-f_max_numpy

f_err_est = np.maximum(np.abs(flow_est)-f_max_numpy,0) # identify violations
f_err_true = np.maximum(np.abs(flow_est0)-f_max_numpy,0)

print(np.max(f_err_est),np.max(f_err_true))
print(np.max(f_err_est/f_max_numpy),np.max(f_err_true/f_max_numpy))

# a= np.asarray([[1,2,1],[2,2,1]])
# b = np.asarray([1,1,1])
# c = (a-b>0)
# print(c)
# print(np.sum(c))

In [ ]:
np.min(f_max_numpy)

In [ ]:
# f_binary_soft = (np.abs(flow_est)-f_max_numpy > 0.01*np.max(f_max_numpy))
# f_binary0_soft = (np.abs(flow_est0)-f_max_numpy > 0.01*np.max(f_max_numpy))
f_binary_soft = (np.abs(flow_est)-f_max_numpy > 0.1*(f_max_numpy))
f_binary0_soft = (np.abs(flow_est0)-f_max_numpy > 0.1*(f_max_numpy))
print(np.sum(f_binary_soft),np.sum(f_binary0_soft))
print(np.sum(f_binary_soft)/f_tot_sample,np.sum(f_binary0_soft)/f_tot_sample)

In [ ]:
f_pred_sort = np.reshape(f_err_est/f_max_numpy,n_line*n_sample)
f_true_sort = np.reshape(f_err_true/f_max_numpy,n_line*n_sample)

# fig2 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(f_pred_sort, bins = 10, facecolor='b', alpha=0.75,label = 'pred. f')
# plt.hist(f_true_sort, bins = 10, facecolor='g', alpha=0.75,label = 'true f')
# plt.legend(loc="upper right")
# plt.xlabel('percentage')
# plt.ylabel('frequency')
# plt.title('flow violation level histogram')
# plt.subplot(1,2,2)
# plt.hist(f_true_sort, bins = 10, facecolor='g', alpha=0.75,label = 'true f')
# plt.legend(loc="upper right")
# plt.xlabel('percentage')
# plt.ylabel('frequency')
# plt.title('flow violation level histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

In [ ]:
f_line = np.sum(f_binary,0)
f_samp = np.sum(f_binary,1)
print('max sample pred:',np.max(f_line))
print('max line pred:',np.max(f_samp))

f_line0  = np.sum(f_binary0,0)
f_samp0 = np.sum(f_binary0,1)
print('max sample true:',np.max(f_line0))
print('max line true:',np.max(f_samp0))

# Check objective optimality

In [ ]:
# gen_pred_binary_full = np.zeros((n_bus,n_sample))
# gen_true_binary_full = np.zeros((n_bus,n_sample))

# p_inj = gen_inj #- load0
# p_inj_true = gen_inj_true #- load0

# gen_cost0 = x[:,4,:].copy()
# # lmp_data = y.copy()

# quadratic_a = x[:,5,:].copy()
print(gen_cost0.shape)

In [ ]:
gen_cost_pred = np.zeros((n_bus,n_sample))
gen_cost_true = np.zeros((n_bus,n_sample))
objective_err = np.zeros(n_sample)

gen_cost_pred = np.multiply(np.multiply(p_inj,p_inj),quadratic_a) + np.multiply(p_inj,gen_cost0)
gen_cost_true = np.multiply(np.multiply(p_inj_true,p_inj_true),quadratic_a) + np.multiply(p_inj_true,gen_cost0)

objective_err = np.sum(np.abs(gen_cost_true-gen_cost_pred),axis=0) / np.sum(gen_cost_true,axis=0)
print(np.mean(objective_err))

# fig6 = plt.figure(figsize=(16, 8))
# # error histogram
# plt.hist(objective_err, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
# # plt.hist(err_linf_new, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
# plt.legend(loc="upper right")
# plt.xlabel('sample error value')
# plt.ylabel('frequency')
# plt.title('Histogram of L_2 error')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

## injection accuracy

In [ ]:
print(p_inj_true.shape,p_inj.shape)

p_inj_true_sort = np.reshape(p_inj_true,n_bus*n_sample)
p_inj_sort = np.reshape(p_inj,n_bus*n_sample)

p_err = np.zeros(n_sample)
for i in range(n_sample):
  p_err[i] = np.linalg.norm(p_inj_true[:,i]-p_inj[:,i]) / np.linalg.norm(p_inj_true[:,i])

print('mean p_inj l2 err:',np.mean(p_err))
# fig3 = plt.figure(figsize=(16, 8))
# plt.subplot(1,2,1)
# plt.hist(p_inj_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. injection')
# plt.hist(p_inj_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true injection')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('injection histogram')
# plt.subplot(1,2,2)
# plt.hist(p_err, bins = 50, facecolor='b', alpha=0.75,label = 'injection err')
# plt.legend(loc="upper right")
# plt.xlabel('value')
# plt.ylabel('frequency')
# plt.title('error histogram')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# plt.grid(True)
# plt.show()

# Test AC feasibility
* P in actual value, V in p.u.
* Use P to recover $\theta$, or solve $\theta$ and Q for PF
$$ Q_m = V_m \sum_{n=1}^N V_n \left(G_{mn}\sin\theta_{mn} - B_{mn}\cos\theta_{mn} \right) $$
calculate $Q_{mn}$ directly

In [ ]:
# Bbus and B_r inverse
filename1 = root+'data_1354ac_2022/ieee1354_Bbus.txt'
Bbus=pd.read_table(filename1,sep=',',header=None).to_numpy()
B_r = np.delete(Bbus,639,axis=0)
B_r = np.delete(B_r,639,axis=1)

Br_inv = np.linalg.inv(B_r)

# Jacobian for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_J1.txt'
J1=pd.read_table(filename1,sep=',',header=None).to_numpy()
# J1_r = np.delete(J1,639,axis=0) # already reduced
# J1_r = np.delete(J1_r,639,axis=1)
J1r_inv = np.linalg.inv(J1)

# nominal angle for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_theta_nominal.txt'
theta_nominal=pd.read_table(filename1,sep=',',header=None).to_numpy()
# nominal injection for fast decoupled power flow 
filename1 = root+'data_1354ac_2022/ieee1354_ieee1354_pinj_nominal.txt'
pinj_nominal=pd.read_table(filename1,sep=',',header=None).to_numpy()


# line parameters
filename1 = root+'data_1354ac_2022/ieee1354_lineloc.txt'
line_loc = pd.read_table(filename1,sep=',',header=None).to_numpy()
# line_entry_loc = np.nonzero(G_mat) # index of nonzero entries
# # not in p.u.
# G_line = np.zeros(n_line)
# B_line = np.zeros(n_line)

# # print(line_loc.shape) #[186,2]
# for i in range(n_line):
#   G_line[i] = G_mat[line_loc[i,0]-1,line_loc[i,1]-1]
#   B_line[i] = B_mat[line_loc[i,0]-1,line_loc[i,1]-1]

# load line params
filename1 = root+'data_1354ac_2022/ieee1354_lineparams_tap.txt'
line_params = pd.read_table(filename1,sep=',',header=None).to_numpy()

n_line = 1751 # the rest are transformers
line_loc = line_loc[:n_line,:].copy()
line_params = line_params[:n_line,:].copy()

# filename2 = 'data_1354ac_2022/ieee1354_Ybus.txt'
# Y_bus = pd.read_table(filename2,sep=',',header=None).to_numpy()

# G_line = line_params[:,0].copy()
# B_line = line_params[:,1].copy()
R_line = line_params[:,0].copy()
X_line = line_params[:,1].copy()

B_shunt = line_params[:,2].copy()

Z_line = R_line + 1j * X_line 
Y_line = 1 / Z_line

G_line = np.real(Y_line)
B_line = np.imag(Y_line)

# transformer indicator
# a = R_line > 0
a = np.ones(n_line)
a=torch.from_numpy(a)
G_line=torch.from_numpy(G_line)
B_line=torch.from_numpy(B_line)
B_shunt=torch.from_numpy(B_shunt)

# P_inj w/out reference bus in p.u.
p_inj_r = np.delete(p_inj,639,axis=0) / 100
p_inj_true_r = np.delete(p_inj_true,639,axis=0) / 100

p_inj_pu = p_inj / 100
p_inj_true_pu = p_inj_true / 100

# print(Br_inv.shape,p_inj.shape,p_inj_true.shape)#p_inj_true
print(J1r_inv.shape,p_inj.shape,p_inj_true.shape)
print(J1r_inv.shape,p_inj_r.shape,pinj_nominal.shape)

# calculate angle (w/our reference at 640th node)
# theta0 = np.matmul(Br_inv,p_inj_r)
# theta_true0 = np.matmul(Br_inv,p_inj_true_r)
theta0 = np.matmul(J1r_inv,p_inj_r-pinj_nominal)
theta_true0 = np.matmul(J1r_inv,p_inj_true_r-pinj_nominal)

theta = np.insert(theta0,639,0,axis = 0)
theta_true = np.insert(theta_true0,639,0,axis = 0)
# restore from deviation
theta = theta + theta_nominal/180*math.pi
theta_true = theta_true + theta_nominal/180*math.pi

print(theta.shape,theta_true.shape)

In [ ]:
print(np.matmul(J1r_inv,p_inj_r-pinj_nominal))

In [ ]:
for i in range(5):
  print(G_line[i],B_line[i])

In [ ]:
print(np.max(theta),np.min(theta))
# math.sin(math.pi/6)
print(G_line[0],B_line[0])

In [ ]:
print(type(G_line))
print(np.max(y_pred1[:,1,:]),np.min(y_pred1[:,1,:]))
print(np.max(theta),np.min(theta))
print(math.sin(math.pi/6))

In [ ]:
# B_line
import math

# n_sample1 = 500
n_sample1 = n_sample
f_tot_sample1 = n_line*n_sample1

# load true voltage file
filename1 = root+'data_1354ac_2022/ieee1354pegase_ac_9974_cases_v_04022022.txt'
v_true = pd.read_table(filename1,sep=',',header=None).to_numpy()
print(v_true.shape)

# load true theta file
filename2 = root+'data_1354ac_2022/ieee1354pegase_ac_9974_cases_theta_04022022.txt'
theta_true = pd.read_table(filename2,sep=',',header=None).to_numpy()
# theta_true = theta_true / 180 * math.pi
print(theta_true)
print(theta_true.shape)

v_true = np.delete(v_true, del_idx,axis=1)
theta_true = np.delete(theta_true, del_idx,axis=1)
print(v_true.shape,theta_true.shape)

In [ ]:
# print(theta)
# print('')
# print(theta_true)
v_pred = y_pred1[:,1,:].copy()
v_pred = v_pred / vy_scale + vy_deviation
print(v_pred)
print('')
print(v_true)

In [ ]:


# Calculate real and reactive flow
f_p = np.zeros((n_line,n_sample1))
f_q = np.zeros((n_line,n_sample1))
fji_p = np.zeros((n_line,n_sample1))
fji_q = np.zeros((n_line,n_sample1))
print(f_q.shape)

v_pred = y_pred1[:,1,:].copy()
v_pred = v_pred / vy_scale + vy_deviation

# v_pred = v_true.copy()
# theta = theta_true.copy()

print(np.max(v_pred),np.min(v_pred),v_pred.shape)

theta1 = theta[line_loc[:,0]-1,:]
theta2 = theta[line_loc[:,1]-1,:]
V1 = torch.from_numpy(v_pred[line_loc[:,0]-1,:]).double()
V2 = torch.from_numpy(v_pred[line_loc[:,1]-1,:]).double() 
theta1=torch.from_numpy(theta1)
theta2=torch.from_numpy(theta2)

f_p=(a*G_line*(V1*V1).T)-a*((V1*V2).T)*(G_line*torch.cos(theta1-theta2).T+B_line*torch.sin(theta1-theta2).T)
f_p=f_p.T
f_q=-a*(V1.T)*(a*V1.T)*(B_line+B_shunt/2)+a*((V1*V2).T)*(B_line*torch.cos(theta1-theta2).T-G_line*torch.sin(theta1-theta2).T)
f_q=f_q.T

theta1 = theta[line_loc[:,1]-1,:]
theta2 = theta[line_loc[:,0]-1,:]
V1 = torch.from_numpy(v_pred[line_loc[:,1]-1,:]).double()
V2 = torch.from_numpy(v_pred[line_loc[:,0]-1,:]).double()
theta1=torch.from_numpy(theta1)
theta2=torch.from_numpy(theta2)

fji_p=(a*G_line*(V1*V1).T)-a*((V1*V2).T)*(G_line*torch.cos(theta1-theta2).T+B_line*torch.sin(theta1-theta2).T)
fji_p=fji_p.T
fji_q=-a*(V1.T)*(a*V1.T)*(B_line+B_shunt/2)+a*((V1*V2).T)*(B_line*torch.cos(theta1-theta2).T-G_line*torch.sin(theta1-theta2).T)
fji_q=fji_q.T

In [ ]:
print('mean p_ij,q_ij:',torch.mean(f_p),torch.mean(f_q))
print('mean p_ji,q_ji:',torch.mean(fji_p),torch.mean(fji_q))
# print(f_q)

In [ ]:
theta = theta_true.copy()
print('Scale check:')
i=0;j=0
# a[0] = 1
a = np.ones(1751)
print('G,B,loc:',G_line[i],B_line[i],line_loc[i,0]-1,line_loc[i,1]-1) # G,B,line_loc verified
print('v:',v_pred[line_loc[i,0]-1,j],v_pred[line_loc[i,1]-1,j]) # v verified
print('theta:',theta[line_loc[i,0]-1,j],theta[line_loc[i,1]-1,j]) # theta verified
print('p,q:',f_p[0,0],f_q[0,0],fji_p[i,j],fji_q[i,j])


f_p_test = a[i] * v_pred[line_loc[i,0]-1,j] * v_pred[line_loc[i,0]-1,j] * G_line[i] \
             - (a[i] * v_pred[line_loc[i,0]-1,j]) * v_pred[line_loc[i,1]-1,j] \
             * (G_line[i] * math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]) \
              + B_line[i] * math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))
             
f_q_test =-(a[i] * v_pred[line_loc[i,0]-1,j]) **2 * (B_line[i] + B_shunt[i]/2) \
             + a[i] * v_pred[line_loc[i,0]-1,j] * v_pred[line_loc[i,1]-1,j] \
             * (B_line[i] * math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]) \
              - G_line[i] * math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))
             
print('test p/q:',f_p_test,f_q_test)

In [ ]:
print(a[i],v_pred[line_loc[i,0]-1,j],B_line[i] + B_shunt[i]/2,v_pred[line_loc[i,1]-1,j])
print(G_line[i],math.cos(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]),math.sin(theta[line_loc[i,0]-1,j] - theta[line_loc[i,1]-1,j]))

In [ ]:
# Calculate apparent power
# s_pred = np.zeros((n_line,n_sample1))
# sji_pred = np.zeros((n_line,n_sample1))

# for i in range(n_line):
#   for j in range(n_sample1):
#     # s_pred[i,j] = np.sqrt(flow_est[i,j]**2 + f_q[i,j]**2)
#     s_pred[i,j] = np.sqrt(f_p[i,j]**2 + f_q[i,j]**2)
#     sji_pred[i,j] = np.sqrt(fji_p[i,j]**2 + fji_q[i,j]**2)

s_pred = torch.sqrt(f_p*f_p+f_q*f_q)*100
sji_pred = torch.sqrt(fji_p*fji_p+fji_q*fji_q)*100
# s_pred = s_pred * 100 # p.u. to mva
# sji_pred = sji_pred * 100 # p.u. to mva

In [ ]:
# np.max(y_pred1[:,1,:]/100+0.8)

f_p = f_p.numpy();f_q = f_q.numpy()
s_pred = s_pred.numpy();sji_pred = sji_pred.numpy()

print(np.max(f_q),np.min(f_q))
flow_est.shape

print(np.mean(s_pred[0,:]),np.mean(f_max_numpy[0]))

In [ ]:
sij_binary = (np.abs(s_pred)-f_max_numpy[:n_line] > 0)
sji_binary = (np.abs(sji_pred)-f_max_numpy[:n_line] > 0)
s_binary = np.maximum(sij_binary,sji_binary)

# sij_binary0 = (np.abs(flow_est0)-f_max_numpy > 0)
print(np.sum(s_binary))#,np.sum(f_binary0))
print('hard violation rate:',np.sum(s_binary)/f_tot_sample1)#,np.sum(f_binary0)/f_tot_sample)

s_binary_soft = (np.abs(s_pred)-f_max_numpy[:n_line] > 0.1*(f_max_numpy[:n_line]))
# s_binary0_soft = (np.abs(flow_est0)-f_max_numpy > 0.1*(f_max_numpy))
print(np.sum(s_binary_soft))#,np.sum(f_binary0_soft))
print(np.sum(s_binary_soft)/f_tot_sample1)#,np.sum(f_binary0_soft)/f_tot_sample)

In [ ]:
# violation level
sij_violation = np.abs(s_pred)-f_max_numpy[:n_line] #/ f_max_numpy
sij_violation_level = np.maximum(sij_violation,0)
sji_violation = np.abs(sji_pred)-f_max_numpy[:n_line] #/ f_max_numpy
sji_violation_level = np.maximum(sji_violation,0)

s_violation_level = np.maximum(sij_violation_level,sji_violation_level)

s_violation_level = np.divide(s_violation_level,f_max_numpy[:n_line])
s_vio_lvl = np.reshape(s_violation_level,n_line*n_sample1)

print('S violation level:')
print('hard:',np.sum(s_binary)/f_tot_sample1)
print('mean:',np.mean(s_vio_lvl))
print('median:',np.median(s_vio_lvl))
print('max:',np.max(s_vio_lvl))
print('std:',np.std(s_vio_lvl))
print('p99:',np.percentile(s_vio_lvl,99))

f_violation = np.abs(flow_est)-f_max_numpy #/ f_max_numpy
f_violation_level = np.maximum(f_violation,0)
f_violation_level = np.divide(f_violation_level,f_max_numpy)
f_vio_lvl = np.reshape(f_violation_level,1991*n_sample)

print('f violation level:')
print('hard:',np.sum(f_binary)/f_tot_sample,np.sum(f_binary0)/f_tot_sample)
print('mean:',np.mean(f_vio_lvl))
print('median:',np.median(f_vio_lvl))
print('max:',np.max(f_vio_lvl))
print('std:',np.std(f_vio_lvl))
print('p99:',np.percentile(f_vio_lvl,99))

# fig4 = plt.figure(figsize=(6,4))
# plt.hist(s_vio_lvl, bins = 50, facecolor='b', alpha=0.75,label = 's violation')
# plt.hist(f_vio_lvl, bins = 50, facecolor='r', alpha=0.75,label = 'f violation')
# plt.legend(loc="upper right")
# plt.xlabel('violation level')
# plt.ylabel('frequency')
# # plt.title('injection histogram')
# plt.show()

In [ ]:
# err_L2_mean = np.mean(err_L2)
# err_Linf_mean = np.mean(err_Linf)
print('Price L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )
print('std:',np.std(err_L2))
# err_L2_mean_v = np.mean(err_L2_v)
# err_Linf_mean_v = np.mean(err_Linf_v)
print('Voltage L2 mean:', err_L2_mean_v,'L_inf mean:', err_Linf_mean_v )
print('std:',np.std(err_L2_v))